# Walmart Kaggle Project

### Authors:  Shijie Wang and Matthew Odom

In [2]:
%ls

README.md              sample_submission.csv  test.csv.zip
Walmart_Script.ipynb   test.csv               train.csv


In [89]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import random

# load and prepare the training dataset for modeling
train = pd.read_csv("train.csv")
train = train.dropna()
train = train.ix[:, (train.columns != 'FinelineNumber') & (train.columns != 'Upc')]

# aggregate ScanCount by DepartmentDescription for each VisitNumber
data_1 = train.pivot_table(index = 'VisitNumber', columns = 'DepartmentDescription', values = 'ScanCount', 
                           aggfunc = np.sum, fill_value=0)
data_1['VisitNumber'] = data_1.index

# get unique Trip Type, VisitNumber, and Weekday
temp = train.drop(train.columns[[3,4]], axis=1)
temp = temp.drop_duplicates()

train = pd.merge(temp, data_1, on = 'VisitNumber')
train = pd.get_dummies(train)

# load and prepare the testing dataset for modeling
test = pd.read_csv("test.csv")
test = test.dropna()
test = test.ix[:, (test.columns != 'FinelineNumber') & (test.columns != 'Upc')]

# aggregate ScanCount by DepartmentDescription for each VisitNumber
data_1 = test.pivot_table(index = 'VisitNumber', columns = 'DepartmentDescription', values = 'ScanCount', 
                           aggfunc = np.sum, fill_value=0)
data_1['VisitNumber'] = data_1.index

# get unique Trip Type, VisitNumber, and Weekday
temp = test.drop(test.columns[[2,3]], axis=1)
temp = temp.drop_duplicates()

test = pd.merge(temp, data_1, on = 'VisitNumber')
test = pd.get_dummies(test)

X_test = test.ix[:,test.columns != 'VisitNumber']
X_test['Intercept'] = 1

# a subset for algorithm build
#data = train[train["TripType"] <= 10]
#data = data.sample(50000)
data = train

# create a loop for each TripType
# stack the output of the test into single submission

TripTypes = data.TripType.unique()
submission = pd.DataFrame()

for i in TripTypes:
    print "Working on TripType " + str(i)
    
    # reset dataset for next iteration
    sample = data
    sample = sample.dropna()
    
    # convert independent variable into binary
    sample.TripType = np.where(sample.TripType != i, 0, 1) 
    sample['Intercept'] = 1
    
    # set up model
    X = sample.ix[:,(sample.columns != 'TripType') & (sample.columns != 'VisitNumber') & (sample.columns != 'HEALTH AND BEAUTY AIDS')]
    y = sample.ix[:,sample.columns == 'TripType']
    
    model = sm.OLS(y,X)
    results = model.fit()
    
    # predict test dataset values
    yhat = pd.DataFrame(results.predict(X_test)) # this should be results.predict(test)
    yhat.columns = ["TripType_%s" % i]
    
    # append columns to submission
    submission = submission.join(yhat, how = "outer")
    
submission

Working on TripType 999
Working on TripType 30
Working on TripType 26
Working on TripType 8
Working on TripType 35
Working on TripType 41
Working on TripType 21
Working on TripType 6
Working on TripType 42
Working on TripType 7
Working on TripType 9
Working on TripType 39
Working on TripType 25
Working on TripType 38
Working on TripType 15
Working on TripType 36
Working on TripType 20
Working on TripType 37
Working on TripType 32
Working on TripType 40
Working on TripType 5
Working on TripType 3
Working on TripType 4
Working on TripType 24
Working on TripType 33
Working on TripType 43
Working on TripType 31
Working on TripType 27
Working on TripType 34
Working on TripType 18
Working on TripType 29
Working on TripType 44
Working on TripType 19
Working on TripType 23
Working on TripType 22
Working on TripType 28
Working on TripType 14
Working on TripType 12


,TripType_999,TripType_30,TripType_26,TripType_8,TripType_35,TripType_41,TripType_21,TripType_6,TripType_42,TripType_7,...,TripType_34,TripType_18,TripType_29,TripType_44,TripType_19,TripType_23,TripType_22,TripType_28,TripType_14,TripType_12
0,0.132898,0.077497,0.003443,0.174057,-0.002392,0.017887,0.003882,0.013936,0.021912,0.040426,...,0.007089,0.002877,0.001831,0.005383,0.002252,0.001340,0.003429,0.003278,-3.248453e-05,0.000766
1,0.106527,0.007726,0.003031,0.178540,0.080542,0.005750,0.003945,0.009799,0.016093,0.066334,...,0.008621,0.009202,0.002267,0.002851,0.001376,0.001134,0.002193,0.004010,4.009902e-05,0.001838
2,0.155996,0.009149,0.004137,0.206422,0.017420,0.001839,0.005693,0.018208,0.004025,0.075890,...,0.008772,0.006007,0.003701,-0.006115,0.003450,0.002080,0.005043,0.004558,5.275011e-06,0.002243
3,0.146221,0.008242,0.003293,0.194836,0.015415,0.001635,0.042990,0.017144,0.017212,0.071707,...,0.008238,0.005011,0.002951,0.001172,0.003133,0.001979,0.004138,0.004217,1.774028e-03,0.001819
4,0.155996,0.009149,0.004137,0.206422,0.017420,0.001839,0.005693,0.018208,0.004025,0.075890,...,0.008772,0.006007,0.003701,-0.006115,0.003450,0.002080,0.005043,0.004558,5.275011e-06,0.002243
5,0.146852,0.008439,0.003909,0.196321,0.034645,0.001079,0.005537,0.015912,0.002044,0.071794,...,0.007316,0.005191,0.002770,-0.006651,0.003289,0.001952,0.005051,0.003927,9.834766e-07,0.003360
6,0.030031,0.005765,-0.002019,0.090758,0.103306,0.010070,-0.002139,-0.001494,0.047998,0.019405,...,0.002576,0.005976,-0.001143,0.026021,-0.000886,0.000315,-0.003701,-0.000243,-7.070601e-05,-0.002198
7,0.054295,0.003813,0.000942,0.172074,0.228498,0.001930,0.039491,0.002052,0.013967,-0.015607,...,0.003491,0.003127,-0.000532,0.008150,0.001762,0.000734,-0.001044,0.001923,1.753762e-03,0.001579
8,0.104874,0.006921,0.002731,0.177517,0.099688,0.002441,0.003615,0.008544,0.005685,0.044555,...,0.005850,0.006345,0.002057,-0.002143,0.002294,0.001343,0.001967,0.003448,1.922250e-05,0.000799
9,0.064747,0.003789,-0.001322,0.098580,0.063467,0.004439,-0.000585,0.003957,0.022728,0.002214,...,0.003866,0.006321,0.000577,0.022778,0.000201,0.000669,-0.002502,0.000523,-1.713634e-06,0.000114


In [98]:
# clean up submission dataset for Kaggle submission
submission = submission.sort_index(axis=1)
print submission.idxmax(axis=1)
submission

0         TripType_8
1         TripType_8
2         TripType_8
3         TripType_8
4         TripType_8
5         TripType_8
6        TripType_25
7        TripType_35
8         TripType_8
9         TripType_5
10        TripType_8
11        TripType_8
12        TripType_8
13        TripType_8
14        TripType_8
15        TripType_8
16        TripType_8
17        TripType_8
18        TripType_8
19        TripType_8
20        TripType_8
21       TripType_39
22        TripType_8
23        TripType_8
24        TripType_8
25        TripType_8
26        TripType_8
27        TripType_8
28        TripType_8
29       TripType_40
            ...     
94258    TripType_27
94259    TripType_31
94260    TripType_40
94261    TripType_22
94262     TripType_8
94263     TripType_8
94264     TripType_8
94265    TripType_32
94266    TripType_39
94267    TripType_24
94268     TripType_8
94269    TripType_40
94270     TripType_8
94271    TripType_24
94272    TripType_24
94273     TripType_8
94274     Tri

,TripType_12,TripType_14,TripType_15,TripType_18,TripType_19,TripType_20,TripType_21,TripType_22,TripType_23,TripType_24,...,TripType_41,TripType_42,TripType_43,TripType_44,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_999
0,0.000766,-3.248453e-05,0.004338,0.002877,0.002252,0.001818,0.003882,0.003429,0.001340,0.013772,...,0.017887,0.021912,0.008931,0.005383,0.028326,0.013936,0.040426,0.174057,0.151459,0.132898
1,0.001838,4.009902e-05,0.022788,0.009202,0.001376,0.002781,0.003945,0.002193,0.001134,0.017497,...,0.005750,0.016093,0.013078,0.002851,0.022596,0.009799,0.066334,0.178540,0.094589,0.106527
2,0.002243,5.275011e-06,0.008778,0.006007,0.003450,0.003439,0.005693,0.005043,0.002080,0.023396,...,0.001839,0.004025,0.007937,-0.006115,0.040590,0.018208,0.075890,0.206422,0.151894,0.155996
3,0.001819,1.774028e-03,0.005911,0.005011,0.003133,0.003366,0.042990,0.004138,0.001979,0.025003,...,0.001635,0.017212,0.010201,0.001172,0.038950,0.017144,0.071707,0.194836,0.147964,0.146221
4,0.002243,5.275011e-06,0.008778,0.006007,0.003450,0.003439,0.005693,0.005043,0.002080,0.023396,...,0.001839,0.004025,0.007937,-0.006115,0.040590,0.018208,0.075890,0.206422,0.151894,0.155996
5,0.003360,9.834766e-07,0.007021,0.005191,0.003289,0.002620,0.005537,0.005051,0.001952,0.021638,...,0.001079,0.002044,0.007874,-0.006651,0.038599,0.015912,0.071794,0.196321,0.143730,0.146852
6,-0.002198,-7.070601e-05,0.056302,0.005976,-0.000886,-0.000959,-0.002139,-0.003701,0.000315,-0.002811,...,0.010070,0.047998,0.013736,0.026021,-0.002924,-0.001494,0.019405,0.090758,0.084785,0.030031
7,0.001579,1.753762e-03,0.007684,0.003127,0.001762,0.000417,0.039491,-0.001044,0.000734,0.013192,...,0.001930,0.013967,0.010908,0.008150,0.011627,0.002052,-0.015607,0.172074,0.050659,0.054295
8,0.000799,1.922250e-05,0.011280,0.006345,0.002294,0.002046,0.003615,0.001967,0.001343,0.015583,...,0.002441,0.005685,0.009363,-0.002143,0.022579,0.008544,0.044555,0.177517,0.097252,0.104874
9,0.000114,-1.713634e-06,0.008050,0.006321,0.000201,0.000669,-0.000585,-0.002502,0.000669,0.088365,...,0.004439,0.022728,0.017726,0.022778,0.242489,0.003957,0.002214,0.098580,0.055893,0.064747


In [151]:
# sample of summary
print results.summary()

                            OLS Regression Results                            
Dep. Variable:               TripType   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     378.1
Date:                Sat, 21 Nov 2015   Prob (F-statistic):               0.00
Time:                        22:47:28   Log-Likelihood:                -5161.5
No. Observations:               48456   AIC:                         1.048e+04
Df Residuals:                   48379   BIC:                         1.115e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------